## 目前的思路是
1. 构造一个函数，输入为json列表，输出为处理好内容的字典列表
2. 针对每一个json
    - {"id": "id-7-0", "id_old": 102623145, "text": "中国医药：子公司两批次药品不符合规定 被药监局罚没457.13万元", "event_list": [{"event_type": "司法行为-罚款", "trigger": "罚没", "arguments": [{"role": "罚款对象", "argument": "中国医药"}, {"role": "执法机构", "argument": "药监局"}, {"role": "罚款金额", "argument": "457.13万元"}]}]}
    - 针对每一个arguments
        - 做一次分词，然后将分词结果同 句子的分词结果做比对，如果argument的分词结果 in 句子的分词结果，就把句子的分词结果替换argument的
        - 然后替换完毕的argumen,去重之后合并

In [7]:
from LAC import LAC

# 装载LAC模型
lac = LAC(mode='lac')

# 单个样本输入，输入为Unicode编码的字符串
text = u"LAC是个优秀的分词工具"
lac_result = lac.run(text)

# 批量样本输入, 输入为多个句子组成的list，平均速率更快
texts = ["当时，在北京市市场监督管理局官网公布的电动车抽检信息中，天津爱玛车业科技有限公司生产的5批次电动自行车存在不合格问题，包括最高车速、脚踏行驶能力、整车质量、反射器和鸣号装置、欠压、过流保护功能等问题",'电动自行车']
texts = ["中国医药：子公司两批次药品不符合规定 被药监局罚没457.13万元"]
lac_result = lac.run(texts)
print(lac_result)


[[['中国', '医药', '：', '子公司', '两批次', '药品', '不符合规定', ' ', '被', '药监局', '罚没', '457.13万元'], ['LOC', 'n', 'w', 'n', 'm', 'n', 'n', 'w', 'p', 'r', 'v', 'm']]]


In [14]:
import json

news = []
with open('output/DuEE1.0/duee_surbot.json', 'r', encoding='utf-8') as fr:
    for new in fr.readlines():
        news.append(json.loads(new))
for new in news: # 针对每一个json
    if len(new['event_list']): # 含有事件
        text = lac.run(new['text'])[0] 
        print(text)
        for event in new['event_list']: # 针对每一个时间
            arguments = []
            for role in event['arguments']: # 针对每一个时间的arguments
                arguments.append(role['argument'])
            arguments_fenci = lac.run(arguments)
            for index_argument in range(len(arguments_fenci)): # 针对每一个argument
                for index_fenci in range(len(arguments_fenci[index_argument][0])): # 针对每一个argument的每一个分词结果
                    res = arguments_fenci[index_argument][0][index_fenci] # 每一个分词结果
                    for index_text_fenci in range(len(text[0])):
                        if res in text[0][index_text_fenci]:
                            if text[1][index_text_fenci] != 'v':
                                arguments_fenci[index_argument][0][index_fenci] = text[1][index_text_fenci]
                            else:
                                pass
            for index_argument in range(len(arguments_fenci)): # 针对每一个argument
                argument = ''.join(arguments_fenci[index_argument][0])
                # argument = ''.join(sorted(arguments_fenci[index_argument][0]), key=arguments_fenci[index_argument][0].index) # 去重
                event['arguments'][index_argument]['argument'] = argument            
a = 1


['中国', '医药', '(', '600056.SH', ')', '子公司', '通用', '康力', '被', '罚没', '457万元']
['中国', '医药', '：', '子公司', '两批次', '药品', '不符合规定', ' ', '被', '药监局', '罚没', '457.13万元']
['新疆维吾尔自治区市场监督管理局', '消息', '，', '针对', '新疆麦趣尔集团股份有限公司', '生产', '的', '纯牛奶', '检出', '丙二醇', '问题', '，', '已', '督促', '企业', '立即', '停止', '纯牛奶', '生产', '，', '及时', '下架', '、', '封存', '、', '召回', '不合格', '产品']
['罚没', '457万', '！', '海南通用康力制药', '深陷', '生产', '劣药', '风波']
['据', '行政处罚决定书', '显示', '，', '因', '生产', '销售', '劣药', '注射用', '奥扎格雷钠', '，', '海南通用康力制药有限公司', '被', '罚没', '4571320.9元']
['北京永辉超市', '因', '牛蛙', '中', '兽药', '残留', '超标', '被', '罚', '5万元']
['北京市', '企业', '信用', '信息网', '显示', '，', '1月10日', '，', '北京永辉超市有限公司', '因', '供应', '的', '牛蛙', '不符合', '食品', '安全', '标准', '中', '兽药', '最大', '残留', '限量', '要求', '，', '被', '北京市石景山区市场监督管理局', '罚款', '5万元', '、', '没收', '违法所得', '129.64元']
['重庆永辉超市有限公司涪陵区中慧西街分公司', '销售', '的', '牛蛙', '抽检', '不合格', ' ', '风险', '控制', '完成', '情况', '公布']
['四川永辉超市', '因', '泥鳅', '兽药残留', '超标', '被', '罚款', '5.5万元']
['据天眼', '查', '信息', '显示', '，', '近日', '，', '四川永辉超市有限公司', '成

IndexError: string index out of range

In [10]:
# a = [1,2,3]
# for i in range(len(a)):
#     a[i] = -1
# print(a)

dic = {"id": "id-7-0", "id_old": 102623145, "text": "中国医药：子公司两批次药品不符合规定 被药监局罚没457.13万元", "event_list": [{"event_type": "司法行为-罚款", "trigger": "罚没", "arguments": [{"role": "罚款对象", "argument": "中国医药"}, {"role": "执法机构", "argument": "药监局"}, {"role": "罚款金额", "argument": "457.13万元"}]}]}
for event in dic["event_list"]:
    for argument in event['arguments']:
        argument['argument'] = -1
print(dic)

{'id': 'id-7-0', 'id_old': 102623145, 'text': '中国医药：子公司两批次药品不符合规定 被药监局罚没457.13万元', 'event_list': [{'event_type': '司法行为-罚款', 'trigger': '罚没', 'arguments': [{'role': '罚款对象', 'argument': -1}, {'role': '执法机构', 'argument': -1}, {'role': '罚款金额', 'argument': -1}]}]}
